In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from datetime import datetime
import time

In [3]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
sns.set()

In [4]:
from math import sqrt

In [5]:
from sklearn.metrics import (r2_score, mean_squared_error, mean_absolute_error)
from sklearn.model_selection import (cross_val_score, cross_val_predict, train_test_split, 
                                     KFold, StratifiedKFold, GridSearchCV)
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, fbeta_score, roc_auc_score, roc_curve, make_scorer
from sklearn.metrics import confusion_matrix

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
import xgboost as xgb

In [9]:
selection = 25
year = '2019'
airline = 'DL'
download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'
file_path = os.path.join(download_path, f'{selection}airports{airline}{year}.csv')

In [10]:
delta_df = pd.read_csv(file_path)

In [11]:
delta_df.columns

Index(['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Total 

In [107]:
delta_df['Scheduled Departure Time'] = delta_df['Scheduled Departure Time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

In [108]:
delta_df['Scheduled Arrival Time'] = delta_df['Scheduled Arrival Time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

In [109]:
delta_df['Total Number of Scheduled Flights'] = delta_df['Total Number of Scheduled Flights'].fillna(0).isna().sum()

In [13]:
X = delta_df[['Origin Airport', 
       'Total Number of Scheduled Flights Region_y',
       'Avg Min Delayed Due To Delta Yesterday Airport',
       'Avg Min Delayed Due To NAS Yesterday Airport',
       'Avg Min Delayed Due To Security Yesterday Airport',
       'Avg Min Delayed Due To Late Arrival Yesterday Airport',
       'Avg Min Delayed Due To Weather Yesterday Airport']]

In [14]:
X = pd.get_dummies(X,columns=['Origin Airport'],drop_first=True)

In [15]:
y = delta_df['Delayed Departure']

In [16]:
delayed = y.sum()
perc_delayed = round(y.sum()/y.count(),2)
print(f'Number of delayed Delta flights in 2019: {delayed}, {int(perc_delayed*100)}% were delayed')

not_delayed = y.count() - delayed 
perc_not_delayed = 1 - perc_delayed
print(f'Number of not delayed Delta flights in 2019: {not_delayed}, {int(perc_not_delayed*100)}% were not delayed')

Number of delayed Delta flights in 2019: 105882, 15% were delayed
Number of not delayed Delta flights in 2019: 598619, 85% were not delayed


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify = y)

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify = y_train)

# Logistic Regression

In [18]:
lr = LogisticRegression(C=1000)

In [19]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"Precision Scores: {cross_val_score(lr, X_train2, y_train2, cv=skfold, scoring = 'precision')}")

/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

Precision Scores: [0.45415778 0.41990291 0.41153082 0.38016529 0.3815261 ]


/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
print(f"Recall Scores: {cross_val_score(lr, X_train2, y_train2, cv=skfold, scoring = 'recall')}")

/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

Recall Scores: [0.01571724 0.0127647  0.01527337 0.01357633 0.01401904]


/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
print(f"F1 Scores: {cross_val_score(lr, X_train2, y_train2, cv=skfold, scoring = make_scorer(fbeta_score, beta = .8))}")

F1 Scores: [0.035839   0.0305077  0.03272118 0.03002109 0.0332023 ]


In [115]:
lr = LogisticRegression(C=1)

In [116]:
print(f"Recall Scores: {cross_val_score(lr, X_train2, y_train2, cv=skfold, scoring = 'recall')}")

/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

Recall Scores: [0.01734061 0.01047739 0.01106766 0.01099388 0.01578986]


/home/desbrium/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Random Forest

In [22]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10)

In [23]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"Precision Scores: {cross_val_score(rf, X_train2, y_train2, cv=skfold, scoring = 'precision')}")

Precision Scores: [0.58498896 0.55661882 0.59533898 0.59610706 0.60869565]


In [24]:
print(f"Recall Scores: {cross_val_score(rf, X_train2, y_train2, cv=skfold, scoring = 'recall')}")

Recall Scores: [0.01800472 0.02597211 0.02279938 0.01933151 0.0170442 ]


In [ ]:
print(f"F1 Scores: {cross_val_score(rf, X_train2, y_train2, cv=skfold, scoring = make_scorer(fbeta_score, beta = .8))}")

In [26]:
rf.fit(X,y)

RandomForestClassifier(max_depth=10)

In [27]:
rf_features_dict = {'Features': X.columns, 'Importance': rf.feature_importances_}
rf_features = pd.DataFrame(rf_features_dict)
rf_features = rf_features.sort_values('Importance', ascending = False)
rf_features

,Features,Importance
4,Avg Min Delayed Due To Late Arrival Yesterday ...,0.249212
1,Avg Min Delayed Due To Delta Yesterday Region,0.193069
2,Avg Min Delayed Due To NAS Yesterday Region,0.192807
0,Total Number of Scheduled Flights Region,0.135406
5,Avg Min Delayed Due To Weather Yesterday Region,0.130111
9,Dest Region_Northeast,0.034944
6,Origin Region_Northeast,0.015468
10,Dest Region_South,0.013070
7,Origin Region_South,0.012130
8,Origin Region_West,0.008602


In [39]:
delta_df.columns

Index(['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Total 

# XGBoost

In [44]:
gbm = xgb.XGBClassifier(
                        n_estimators=100,
                        max_depth = 3,
                        learning_rate=.1, #lowering learning_rate,
                        early_stopping_rounds=20
                        )

In [45]:
print(f"Precision Scores: {cross_val_score(gbm, X_train2, y_train2, cv=skfold, scoring = 'precision')}")

/home/desbrium/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:13:58] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:13:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:14:01] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:14:01] WARNING: ../src/learner.cc:1095: Starting in XGBoos

In [117]:
print(f"Recall Scores: {cross_val_score(rf, X_train2, y_train2, cv=skfold, scoring = 'recall')}")

Recall Scores: [0.0050915  0.00782115 0.00634546 0.00442706 0.00553383]
